<a href="https://colab.research.google.com/github/raulviana/INESC-summerInternship/blob/main/script_extract_frames_from_videos_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script para extrair frames dos videos
## O fps dos videos está a ser reduzido para metade

In [6]:
import torch, torchvision
import torchvision.transforms as transforms
import cv2
import os
from os import path
import json

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from distutils.dir_util import copy_tree

# copy subdirectory example
fromDirectory = "/content/drive/MyDrive/hmdb51_org"
toDirectory = "/home/dataset"

copy_tree(fromDirectory, toDirectory, verbose=True)

In [ ]:


def extract_frames(file_path, target_dir):
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    vidcap = cv2.VideoCapture(file_path)
    success, image = vidcap.read()
    new_fps = int(vidcap.get(cv2.CAP_PROP_FPS) * 0.4)
    if new_fps != 0 or new_fps:
      ratio = int(vidcap.get(cv2.CAP_PROP_FPS) // new_fps)
    else:
      ratio = 10

    count = 0
    frame_count = 0
    while success:
      frame_path = os.path.join(target_dir, f'{count}.jpg')
      if frame_count % ratio == 0 :
        cv2.imwrite(frame_path, image)     # save frame as JPEG file
        success, image = vidcap.read()
        count += 1
      frame_count += 1


root = '/home/dataset'
frame_root = '/home/hmdb51_org_frames'
labels_path = '/home/hmdb51_org_frames/labels.csv'
class_name_to_label_path = '/home/hmdb51_org_frames/class_name_to_label.json'

# read files
files = []

for class_name in os.listdir(root):
    for video_name in os.listdir(os.path.join(root, class_name)):
        files.append([os.path.join(class_name, video_name), class_name])

# normalize labels
class_name_to_label = {}
current_label = -1

for vid in files:
    label = class_name_to_label.get(vid[1], -1)

    if label == -1:
        current_label += 1
        class_name_to_label[vid[1]] = current_label
        label = current_label

    vid[1] = label


# save file paths
if not os.path.exists(os.path.split(labels_path)[0]):
    os.makedirs(os.path.split(labels_path)[0])

f = open(labels_path, 'w')

f.write('path,label\n')

for vid in files:
    f.write(f'{vid[0]},{vid[1]}\n')

f.close()

# save label normalization
if not os.path.exists(os.path.split(class_name_to_label_path)[0]):
    os.makedirs(os.path.split(class_name_to_label_path)[0])

with open(class_name_to_label_path, 'w') as json_file:
    json.dump(class_name_to_label, json_file, indent=4)

# extract frames
for i, vid in enumerate(files):
    file_path = os.path.join(root, vid[0])
    target_dir = os.path.join(frame_root, vid[0])

    extract_frames(file_path, target_dir)

    print(f'{i+1}/{len(files)}')

In [ ]:
# copy subdirectory example


copy_tree('/home/hmdb51_org_frames', '/content/drive/MyDrive/hmdb51_org_frames', verbose=True)